In [1]:
import pandas as pd
import numpy as np
import sklearn.preprocessing
import sklearn.metrics
import torch
from torch.autograd import Variable

In [3]:
### read in 1st X rows of big data
data = pd.read_csv('../data/assembled_data.csv', nrows = 2000000-380)
data.head()

KeyboardInterrupt: 

In [3]:
### read in census data as example static variables
census = pd.read_csv('../data/sensor_locations_with_census.csv')
site_lat_long = census.loc[:, ('Site_ID', 'Lat', 'Lon')]
census_reduced = census.iloc[:, 8:-1]
census_reduced = pd.concat([site_lat_long, census_reduced], axis = 1)
census_reduced.head()

,Site_ID,Lat,Lon,Population,Land_Sq_Mi,Population_Density,White,Black,Native,Asian,...,Age_60_p,Age_70_p,Income_0_p,Income_25_p,Income_50_p,Income_75_p,Income_100_p,Income_150_p,Income_200_p,Family_p
0,1,30.49748,-87.88026,32285.0,73.738,437.8,28378.0,1901.0,43.0,213.0,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.030,0.716
1,2,33.28493,-85.80361,5195.0,144.453,36.0,4457.0,504.0,42.0,0.0,...,0.114,0.136,0.171,0.124,0.061,0.030,0.025,0.002,0.002,0.687
2,3,34.76262,-87.63810,16861.0,55.916,301.5,13570.0,2666.0,110.0,64.0,...,0.107,0.134,0.095,0.098,0.084,0.052,0.050,0.018,0.005,0.706
3,4,34.28857,-85.96986,9691.0,78.702,123.1,6789.0,4.0,52.0,22.0,...,0.054,0.111,0.109,0.086,0.056,0.022,0.017,0.002,0.002,0.738
4,5,33.99149,-85.99265,17106.0,67.416,253.7,10507.0,5395.0,9.0,16.0,...,0.134,0.102,0.154,0.114,0.069,0.026,0.024,0.007,0.008,0.607


In [4]:
data_final = data.merge(right = census_reduced, left_on = 'site', right_on = 'Site_ID', how = 'left')
data_final = data_final.drop(['Site_ID'], axis = 1)
data_final.head()

,site,year,date,MonitorData,GFEDFireCarbon,USElevation_dsc10000,USElevation_max100,USElevation_max10000,USElevation_mea100,USElevation_mea10000,...,Age_60_p,Age_70_p,Income_0_p,Income_25_p,Income_50_p,Income_75_p,Income_100_p,Income_150_p,Income_200_p,Family_p
0,1,2000,2000-01-01,NaN,0.001167,26.790501,43,30.143499,36.0,26.504299,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.03,0.716
1,1,2000,2000-01-02,NaN,0.001236,26.790501,43,30.143499,36.0,26.504299,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.03,0.716
2,1,2000,2000-01-03,NaN,0.001305,26.790501,43,30.143499,36.0,26.504299,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.03,0.716
3,1,2000,2000-01-04,NaN,0.001373,26.790501,43,30.143499,36.0,26.504299,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.03,0.716
4,1,2000,2000-01-05,NaN,0.001442,26.790501,43,30.143499,36.0,26.504299,...,0.142,0.142,0.061,0.084,0.073,0.047,0.054,0.022,0.03,0.716


In [5]:
def split_sizes_site(sites):
    """Gets the split sizes to split dataset by site for a dataset with multiple sites.
    
    Arguments:
        sites (array): array indicating the site of each row 
    """
    split_sizes = []
    for i in range(len(sites)):
        if i == 0:
            site = sites[i]
            split_sizes.append(i)
        elif site != sites[i]:
            site = sites[i]
            split_sizes.append(i - (len(split_sizes)-1)*split_sizes[len(split_sizes)-1])
        elif i == len(sites)-1:
            split_sizes.append((i+1) - (len(split_sizes)-1)*split_sizes[len(split_sizes)-1])
    
    split_sizes = split_sizes[1:]
    return split_sizes


def split_data(tensor, split_sizes, dim=0):
    """Splits the tensor according to chunks of split_sizes.
    
    Arguments:
        tensor (Tensor): tensor to split.
        split_sizes (list(int)): sizes of chunks
        dim (int): dimension along which to split the tensor.
    """
    if dim < 0:
        dim += tensor.dim()
    
    dim_size = tensor.size(dim)
    if dim_size != torch.sum(torch.Tensor(split_sizes)):
        raise KeyError("Sum of split sizes exceeds tensor dim")
    
    splits = torch.cumsum(torch.Tensor([0] + split_sizes), dim=0)[:-1]
    return tuple(tensor.narrow(int(dim), int(start), int(length)) 
        for start, length in zip(splits, split_sizes))


def pad_stack_splits(site_tuple, split_sizes, x_or_y):
    """Zero (x) or nan (y) pads site data sequences and stacks them into a matrix.
    
    Arguments:
        site_tuple (tuple): tuple of site data sequences to pad and stack
        split_sizes (array): lengths of site data sequences
        x_or_y (string): 'x' or 'y' indicating whether to pad and stack x or y
    """
    data_padded_list = []
    for sequence in site_tuple:
        max_sequence_length = torch.max(torch.from_numpy(split_sizes))

        if x_or_y == 'x':
            zero_padding_rows = torch.zeros(max_sequence_length - sequence.size()[0], sequence.size()[1])
            data_padded_list.append(torch.cat((sequence, zero_padding_rows), dim = 0))
            
        elif x_or_y == 'y':
            nan_padding = torch.zeros(max_sequence_length - sequence.size()[0]).double() * np.nan
            data_padded_list.append(torch.cat((sequence, nan_padding), dim = 0))
            
    return torch.stack(data_padded_list, dim = 0)


def get_monitorData_indices(sequence):
    """Gets indices for a site data sequence for which there is an output for MonitorData.
    
    Arguments:
        sequence (tensor): sequence of MonitorData outputs for a given site, including NaNs
    """
    response_indicator_vec = sequence == sequence
    num_responses = torch.sum(response_indicator_vec)
    response_indices = torch.sort(response_indicator_vec, dim = 0, descending = True)[1][:num_responses]
    ordered_response_indices = torch.sort(response_indices)[0]
    return ordered_response_indices


def r2(model, batch_size, x_stack_vary, x_tuple_static, y_tuple):
    """Computes R-squared
    
    Arguments:
        model (torch): model to test
        batch_size (int): to determine how many sequences to read in at a time
        x_stack (tensor): stack of site data sequences
        y_tuple (tuple): tuple of true y values by sequence, including NaNs
    
    """
    y = []
    pred = []
    
    # get number of batches
    if x_stack_vary.size()[0] % batch_size != 0:
        num_batches = int(np.floor(x_stack_vary.size()[0]/batch_size) + 1)
    else:
        num_batches = int(x_stack_vary.size()[0]/batch_size)
        
    for batch in range(num_batches):
        # get x and y for this batch
        x_stack_batch_vary = x_stack_vary[batch_size * batch:batch_size * (batch+1)]
        x_tuple_batch_static = x_tuple_static[batch_size * batch:batch_size * (batch+1)]
        y_tuple_nans = y_tuple[batch_size * batch:batch_size * (batch+1)]
        
        # get indices for monitor data and actual monitor data
        y_by_site = []
        x_static_by_site = []
        y_ind_by_site = []
        for i in range(len(y_tuple_nans)):
            y_ind = get_monitorData_indices(y_tuple_nans[i])
            y_by_site.append(y_tuple_nans[i][y_ind])
            y_ind_by_site.append(y_ind)
            x_static_by_site.append(x_tuple_batch_static[i][y_ind])
        y_batch = list(Variable(torch.cat(y_by_site, dim=0)).data.numpy())
        x_batch_static = Variable(torch.cat(x_static_by_site, dim=0)).float()
        
        # get model output
        pred_batch = list(cnn(x_stack_batch_vary, x_batch_static, y_ind_by_site).data.numpy())
        
        # concatenate new predictions with ones from previous batches
        y += y_batch
        pred += pred_batch
        
    return sklearn.metrics.r2_score(y, pred)

In [19]:
### CNN model architecture
class CNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_static, kernel_size, padding):
        super(CNN, self).__init__()
        
        self.conv1d = torch.nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size, padding=padding, bias=bias)
        self.norm1 = torch.nn.BatchNorm1d(num_features = hidden_size)
        self.tanh = torch.nn.Tanh()
        #self.norm2 = torch.nn.BatchNorm1d(num_features = hidden_size + num_static)
        self.linear = torch.nn.Linear(in_features = hidden_size + num_static, out_features = 1, bias = True)
        
    def forward(self, input_vary, input_static, y_ind_by_site):
        hidden = self.conv1d(input_vary)
        hidden = self.norm1(hidden)
        hidden = self.tanh(hidden)
        
        hidden_w_response = []
        
        for i in range(hidden.size()[0]):
            hidden_w_response.append(torch.transpose(hidden[i][:, y_ind_by_site[i]], 0, 1)) 
        hidden_w_response = torch.cat(hidden_w_response, dim = 0)
        
        hidden_w_response__input_static = torch.cat([hidden_w_response, input_static], dim = 1)
        
        #hidden = self.norm2(hidden_w_response__input_static)
        output = self.linear(hidden_w_response__input_static)

        return output

In [7]:
### train/val/test split by site id
np.random.seed(1)

# get sites for val/test data
val_test_sites = np.random.choice(np.unique(data_final['site'].values), round(len(np.unique(data_final['site'].values))/4), replace = False)

# get sites for test data
test_sites = np.random.choice(np.unique(val_test_sites), round(len(np.unique(val_test_sites))/2), replace = False)

# train sites/rows and x/y split
train = data_final[~data_final['site'].isin(val_test_sites)]
train_x = train.iloc[:, 4:]
train_y = train.loc[:, 'MonitorData']
train_sites = train.loc[:, 'site']

# val sites/rows and x/y split
val = data_final[(data_final['site'].isin(val_test_sites)) & (~data_final['site'].isin(test_sites))]
val_x = val.iloc[:, 4:]
val_y = val.loc[:, 'MonitorData']
val_sites = val.loc[:, 'site']

# test sites/rows and x/y split
test = data_final[data_final['site'].isin(test_sites)]
test_x = test.iloc[:, 4:]
test_y = test.loc[:, 'MonitorData']
test_sites = test.loc[:, 'site']

In [8]:
### impute mean
mean_imputer = sklearn.preprocessing.Imputer(strategy = 'mean')
train_x_imp = mean_imputer.fit_transform(train_x)
val_x_imp = mean_imputer.transform(val_x)
test_x_imp = mean_imputer.transform(test_x)

In [9]:
### standardize features
standardizer = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
train_x_imp_std = standardizer.fit_transform(train_x_imp)
val_x_imp_std = standardizer.transform(val_x_imp)
test_x_imp_std = standardizer.transform(test_x_imp)

In [10]:
### break x into varying and static variables
train_x_imp_std_vary = train_x_imp_std[:, :112]
train_x_imp_std_static = train_x_imp_std[:, 112:]

val_x_imp_std_vary = val_x_imp_std[:, :112]
val_x_imp_std_static = val_x_imp_std[:, 112:]

test_x_imp_std_vary = test_x_imp_std[:, :112]
test_x_imp_std_static = test_x_imp_std[:, 112:]

In [11]:
### get split sizes for TRAIN data (splitting by site)
train_split_sizes = split_sizes_site(train_sites.values)

### get tuples by site
train_x_std_tuple_vary = split_data(torch.from_numpy(train_x_imp_std_vary).float(), train_split_sizes, dim = 0)
train_x_std_tuple_static = split_data(torch.from_numpy(train_x_imp_std_static).float(), train_split_sizes, dim = 0)
train_y_tuple = split_data(torch.from_numpy(train_y.values), train_split_sizes, dim = 0)


### get site sequences stacked into matrix to go through CNN
train_x_std_stack_vary = pad_stack_splits(train_x_std_tuple_vary, np.array(train_split_sizes), 'x')
train_x_std_stack_vary = Variable(torch.transpose(train_x_std_stack_vary, 1, 2))


### get split sizes for VALIDATION data (splitting by site)
val_split_sizes = split_sizes_site(val_sites.values)

### get tuples by site
val_x_std_tuple_vary = split_data(torch.from_numpy(val_x_imp_std_vary).float(), val_split_sizes, dim = 0)
val_x_std_tuple_static = split_data(torch.from_numpy(val_x_imp_std_static).float(), val_split_sizes, dim = 0)
val_y_tuple = split_data(torch.from_numpy(val_y.values), val_split_sizes, dim = 0)

### get site sequences stacked into matrix to go through CNN
val_x_std_stack_vary = pad_stack_splits(val_x_std_tuple_vary, np.array(val_split_sizes), 'x')
val_x_std_stack_vary = Variable(torch.transpose(val_x_std_stack_vary, 1, 2))


### get split sizes for TEST data (splitting by site)
test_split_sizes = split_sizes_site(test_sites.values)

### get tuples by site
test_x_std_tuple_vary = split_data(torch.from_numpy(test_x_imp_std_vary).float(), test_split_sizes, dim = 0)
test_x_std_tuple_static = split_data(torch.from_numpy(test_x_imp_std_static).float(), test_split_sizes, dim = 0)
test_y_tuple = split_data(torch.from_numpy(test_y.values), test_split_sizes, dim = 0)

### get site sequences stacked into matrix to go through CNN
test_x_std_stack_vary = pad_stack_splits(test_x_std_tuple_vary, np.array(test_split_sizes), 'x')
test_x_std_stack_vary = Variable(torch.transpose(test_x_std_stack_vary, 1, 2))

In [22]:
# CNN parameters
input_size = train_x_imp_std_vary.shape[1]
hidden_size = 25
kernel_size = 3
num_static = train_x_imp_std_static.shape[1]
padding = 1
bias = True

# instantiate model
cnn = CNN(input_size, hidden_size, num_static, kernel_size, padding)

# Loss function
mse_loss = torch.nn.MSELoss(size_average=True)

# Optimizer
lr = 0.0000001
weight_decay = 0.000001
optimizer = torch.optim.SGD(cnn.parameters(), lr=lr, weight_decay=weight_decay)

In [23]:
num_epochs = 10000
batch_size = 15

# get number of batches
if train_x_std_stack_vary.size()[0] % batch_size != 0:
    num_batches = int(np.floor(train_x_std_stack_vary.size()[0]/batch_size) + 1)
else:
    num_batches = int(train_x_std_stack_vary.size()[0]/batch_size)
    
    
for epoch in range(num_epochs):
    epoch_loss = 0
    
    for batch in range(num_batches):
        # get x and y for this batch
        x_stack_batch_vary = train_x_std_stack_vary[batch_size * batch:batch_size * (batch+1)]
        x_tuple_batch_static = train_x_std_tuple_static[batch_size * batch:batch_size * (batch+1)]
        y_tuple_nans = train_y_tuple[batch_size * batch:batch_size * (batch+1)]
        
        # get indices for monitor data and actual monitor data
        y_by_site = []
        x_static_by_site = []
        y_ind_by_site = []
        for i in range(len(y_tuple_nans)):
            y_ind = get_monitorData_indices(y_tuple_nans[i])
            y_by_site.append(y_tuple_nans[i][y_ind])
            y_ind_by_site.append(y_ind)
            x_static_by_site.append(x_tuple_batch_static[i][y_ind])
        y_batch = Variable(torch.cat(y_by_site, dim=0)).float()
        x_batch_static = Variable(torch.cat(x_static_by_site, dim=0)).float()
        
        # get model output
        pred_batch = cnn(x_stack_batch_vary, x_batch_static, y_ind_by_site)
        
        # compute loss, backprop, and update parameters
        loss_batch = mse_loss(pred_batch, y_batch)
        loss_batch.backward()
        optimizer.step()
        
        # accumulate loss over epoch
        epoch_loss += loss_batch.data[0]
        
    print(r2(cnn, batch_size, val_x_std_stack_vary, val_x_std_tuple_static, val_y_tuple))
    print(epoch_loss)

-1.69481926025
2908.594757080078
-1.6943237561
2908.1898880004883
-1.69351382731
2907.3789291381836
-1.69239009562
2906.16357421875
-1.69095339252
2904.550163269043
-1.68920479359
2902.529571533203
-1.68714561185
2900.118309020996
-1.68477738276
2897.3028259277344
-1.68210189601
2894.0870666503906
-1.67912116219
2890.486671447754
-1.67583742446
2886.4955444335938
-1.6722532228
2882.1145401000977
-1.66837127985
2877.3556747436523
-1.66419461763
2872.213668823242
-1.65972648052
2866.7030868530273
-1.65497034021
2860.8148498535156
-1.64992994235
2854.5731201171875
-1.64460922955
2847.966537475586
-1.63901236758
2841.006446838379
-1.63314373044
2833.7011795043945
-1.62700785403
2826.05379486084
-1.62060941297
2818.0746536254883
-1.61395319563
2809.76171875
-1.60704405077
2801.1268768310547
-1.59988692087
2792.1767349243164
-1.59248668915
2782.9169845581055
-1.58484827492
2773.357681274414
-1.57697645767
2763.505500793457
-1.56887594687
2753.368453979492
-1.5605513148
2742.947090148926
-1.5

0.0559215188366
1068.2627754211426
0.0781977376728
1044.1876792907715
0.0998410163751
1020.5391273498535
0.12081149772
997.3627395629883
0.141073066363
974.6974105834961
0.160592982262
952.5884132385254
0.179341371299
931.0674571990967
0.197292404768
910.1729602813721
0.214422585974
889.9318923950195
0.23071211272
870.3746013641357
0.246144349788
851.5278396606445
0.260705688265
833.4106101989746
0.27438503201
816.046558380127
0.287174913554
799.4483985900879
0.299070056917
783.6330642700195
0.310068236807
768.6131210327148
0.32016994889
754.3951969146729
0.329377593975
740.9857368469238
0.337696271253
728.3894100189209
0.345133315432
716.6086730957031
0.351698180116
705.6421127319336
0.357402161285
695.4885158538818
0.362258531082
686.144947052002
0.366282247869
677.599853515625
0.369489947011
669.8493614196777
0.371899557936
662.8813953399658
0.373530641742
656.6858291625977
0.374403811984
651.2485790252686
0.374540949344
646.5565567016602
0.373964893532
642.5930309295654
0.372699450

0.406840539664
693.637861251831
0.402567270091
698.6285743713379
0.398187599372
703.6648693084717
0.393716764618
708.7292709350586
0.389169468214
713.8081150054932
0.384561060867
718.8838634490967
0.37990642891
723.9401702880859
0.375220616727
728.96364402771
0.370518222202
733.936653137207
0.365813733979
738.8446159362793
0.36112147599
743.6722621917725
0.356455222977
748.4061660766602
0.351828467118
753.0295791625977
0.347254510946
757.5324172973633
0.342745829038
761.8997421264648
0.338314445592
766.1174755096436
0.333971730881
770.1754894256592
0.329728826736
774.0616149902344
0.325595632138
777.7654094696045
0.321581615312
781.278980255127
0.317695316799
784.5894794464111
0.31394478388
787.6921787261963
0.310336834291
790.578218460083
0.306878290236
793.2406520843506
0.303574218934
795.6752433776855
0.300428970108
797.8769664764404
0.297446419177
799.8393249511719
0.294628977664
801.564432144165
0.291978715679
803.0477085113525
0.289496652558
804.2892818450928
0.287182576163
805.2

0.346443526785
634.0768947601318
0.343826086238
633.6427783966064
0.341268333001
633.1950340270996
0.338781625275
632.7229080200195
0.336377433013
632.2237319946289
0.334066491059
631.6909217834473
0.331858629363
631.1198425292969
0.329763761821
630.5030403137207
0.327790742582
629.8402576446533
0.325947980437
629.1244430541992
0.324242457478
628.3553485870361
0.322681139547
627.5307769775391
0.321269635371
626.6478290557861
0.320012708937
625.7084369659424
0.318914301328
624.7085800170898
0.317977165704
623.6540298461914
0.317202990273
622.5446929931641
0.316592569286
621.3839244842529
0.316145889152
620.1758556365967
0.315861681904
618.9219989776611
0.315737454107
617.633279800415
0.315770305453
616.3106460571289
0.315956289921
614.9628372192383
0.316289892032
613.5957317352295
0.31676524076
612.2188148498535
0.317375116522
610.8415813446045
0.318111845493
609.4724941253662
0.318967347917
608.1196956634521
0.319932383212
606.7963047027588
0.320997000862
605.5092754364014
0.3221506458

0.399492033583
626.490083694458
0.397300896281
629.6771793365479
0.395263571608
632.726095199585
0.393386152873
635.626392364502
0.39167463813
638.3729343414307
0.390131493324
640.953239440918
0.388759160142
643.3627796173096
0.387559831121
645.5941352844238
0.386534898428
647.6431751251221
0.385682575648
649.5061702728271
0.385001261118
651.1823291778564
0.38448887042
652.669319152832
0.384142602196
653.966926574707
0.383957213422
655.0774478912354
0.383927389424
656.0019607543945
0.384046766541
656.7458477020264
0.38430854043
657.3120975494385
0.384705147578
657.7070121765137
0.385228242041
657.9371948242188
0.385868784673
658.0093250274658
0.386617542325
657.9318408966064
0.387464479762
657.7136344909668
0.388399336116
657.365514755249
0.389411287834
656.8954677581787
0.390489734678
656.3151569366455
0.391623961864
655.6366729736328
0.392802544408
654.8683815002441
0.394014868365
654.0260639190674
0.395250224598
653.1184406280518
0.396497527008
652.1580448150635
0.397746601241
651.1

...

0.471992881232
493.3701992034912
0.469324835447
495.6852207183838
0.466485611324
498.17792797088623
0.463488575999
500.84019947052
0.460347477889
503.6585626602173
0.457090353296
506.61863708496094
0.453726351706
509.69741916656494
0.450267522514
512.8898906707764
0.446731199246
516.1802101135254
0.443144350746
519.5521392822266
0.439516288487
522.9854345321655
0.43585674004
526.4714994430542
0.432200377404
529.9952087402344
0.428550269518
533.5340347290039
0.424918133844
537.0810928344727
0.421335041281
540.6228342056274
0.417805767095
544.1343603134155
0.414343464755
547.6154260635376
0.410976105614
551.0411243438721
0.407697916243
554.4038734436035
0.404537319316
557.6960802078247
0.401499617755
560.8944425582886
0.398592021251
563.999345779419
0.395840392115
566.9933738708496
0.393234153507
569.8683776855469
0.390799954527
572.6196575164795
0.388535319763
575.2318859100342
0.386449050586
577.706916809082
0.384552531377
580.026050567627
0.382834787331
582.1955795288086
0.38132288617

0.49908542086
584.9869766235352
0.498079152716
586.6188945770264
0.496999032951
588.1871719360352
0.49585126896
589.6839752197266
0.494640590978
591.1018905639648
0.493372712433
592.4369926452637
0.492054341742
593.6793403625488
0.490690382261
594.8210258483887
0.489287849616
595.8601970672607
0.4878530623
596.7855052947998
0.486391789472
597.5934352874756
0.484913592371
598.2779140472412
0.483427048946
598.8273658752441
0.48193415978
599.2403964996338
0.480443055473
599.5137462615967
0.478958880519
599.643762588501
0.477489357208
599.6237316131592
0.476047971369
599.4473285675049
0.474633105842
599.1129703521729
0.473250866457
598.6194038391113
0.471907620195
597.9662170410156
0.470614155152
597.149393081665
0.469375139762
596.1635780334473
0.468189639283
595.0140113830566
0.467063090144
593.70263671875
0.466007778362
592.2275218963623
0.465024575489
590.5853290557861
0.464111324293
588.7842712402344
0.463277399966
586.8286762237549
0.462528267862
584.7128658294678
0.461858304065
582.

0.478228077967
491.2125463485718
0.479074710813
488.68009090423584
0.47974785723
486.3906316757202
0.480243864077
484.3497486114502
0.480558876186
482.56701278686523
0.480691674448
481.0421886444092
0.480639708585
479.78339862823486
0.480403038259
478.7904510498047
0.479981270459
478.0656051635742
0.479373914644
477.6091070175171
0.478584300768
477.4214162826538
0.477612330748
477.49949073791504
0.476462457214
477.8416919708252
0.475137680853
478.44432735443115
0.473640997905
479.3009376525879
0.471970936654
480.4033031463623
0.470141279086
481.74785900115967
0.468156972326
483.32631397247314
0.466025877188
485.1314105987549
0.463754798548
487.1536560058594
0.461351113278
489.3822717666626
0.458822624494
491.8065137863159
0.456172841589
494.4117126464844
0.453416712442
497.18743991851807
0.450563155403
500.1211271286011
0.447622059704
503.20112133026123
0.444602743569
506.4124364852905
0.441512510512
509.7401342391968
0.438363660736
513.1662855148315
0.43516732215
516.6806535720825
0.4

0.371329602208
581.3083629608154
0.370079615463
582.5199184417725
0.368888487039
583.6504802703857
0.367755480189
584.705436706543
0.366680042963
585.6854858398438
0.365660204174
586.5970439910889
0.364695591651
587.4384956359863
0.36378460893
588.2148494720459
0.36292647634
588.9257183074951
0.362118792743
589.5774173736572
0.361360648003
590.1700191497803
0.360651479629
590.7086029052734
0.35998897307
591.1946697235107
0.359372484743
591.6309661865234
0.3588015247
592.0223751068115
0.358274764075
592.3690013885498
0.357791364474
592.6744556427002
0.357348957304
592.9431324005127
0.356948420256
593.1759052276611
0.35658955436
593.3741836547852
0.35627044957
593.5438537597656
0.355991865819
593.6830463409424
0.355752709556
593.7975921630859
0.355553083707
593.886962890625
0.35539338164
593.953685760498
0.355280202069
593.9974708557129
0.355213772413
594.0185775756836
0.35518704158
594.0184192657471
0.355203195078
593.999345779419
0.355260544992
593.963680267334
0.355360999095
593.91063

KeyboardInterrupt: 

In [736]:
train_x = train_x.loc[train_y.dropna(axis = 0).index, :]
test_x = test_x.loc[test_y.dropna(axis = 0).index, :]

### impute mean
mean_imputer = sklearn.preprocessing.Imputer(strategy = 'mean')
train_x_imp = mean_imputer.fit_transform(train_x)
test_x_imp = mean_imputer.transform(test_x)

train_x_imp_std = standardizer.fit_transform(train_x_imp)
train_y = train_y.dropna(axis = 0).values

test_x_imp_std = standardizer.transform(test_x_imp)
test_y = test_y.dropna(axis = 0).values

In [738]:
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.ensemble

# get ridge coefficients
ridge = sklearn.linear_model.Ridge(alpha = 1000, random_state = 1)
ridge.fit(train_x_imp_std, train_y)
ridge.score(test_x_imp_std, test_y)

0.67064558617874481